In [1]:
import pandas as pd
import pandas_profiling as profiling
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
data_path = './data/data_1.csv'
df = pd.read_csv(data_path)

In [ ]:
profile = profiling.ProfileReport(df)
# profile.to_file("./temp/profile.html")

In [ ]:
df.columns

In [3]:
df.columns = ['patient_no', 'gender', 'year_of_education', 'age_of_onset', 'group', 'ddyv', 'hyv', 'leddv',
       'mmsev', 'updrsv', 'reytimev', 'reyscorev', 'analogyv', 'strooptimev',
       'stroopcorrectv', 'bostonv', 'reyrecallv', 'sdmt1v', 'sdmt4v',
       'avltsumv', 'avltshortv', 'avltlongv', 'avltcuev', 'avltrecognitionv',
       'tmt2numberv', 'tmt2timev', 'vft1v', 'vft2v', 'vft3v', 'cdtv', 'agev',
       'visit_time']

In [4]:
df.drop(['patient_no'], axis=1, inplace=True)

In [ ]:
df['gender'][df['gender'] == 0] = 'Female'
df['gender'][df['gender'] == 1] = 'Male'

df['group'][df['group'] == 0] = 'GU-EOPD (SD)'
df['group'][df['group'] == 1] = 'PD-Parkin (SD)'

In [5]:
target_name = 'updrsv'

X_full = df.copy()

# drop all rows with NaN target
X_full.dropna(axis=0, subset=[target_name], inplace=True)

y_full = X_full[target_name]
X_full.drop([target_name], axis=1, inplace=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=10)

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=5)
model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [8]:
# X_full.to_csv('./temp/X_full.csv')